#### Produce cleaning data

In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import geohash_hilbert as ghh

DEVELOPE = False

### Step1 download and read data
Save a smaller sample when developing

#### cells above should be resumed when download original data

## read data

In [2]:
#downloaded from TLC by month
df = pd.read_csv('data/yellow_tripdata_2015-04.csv', index_col=0)
print ("Shape of data\n{}".format(df.shape))
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Shape of data
(13071789, 18)


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
VendorID,,,,,,,,,,,,,,,,,,
2,2015-04-27 15:16:29,2015-04-27 15:26:14,1,1.05,-73.988869,40.758396,1,N,-73.984779,40.748272,2,7.5,0.0,0.5,0.00,0.0,0.3,8.30
2,2015-04-27 15:16:29,2015-04-27 15:25:03,1,2.67,-74.008568,40.704479,1,N,-73.978271,40.720543,1,10.5,0.0,0.5,1.00,0.0,0.3,12.30
2,2015-04-27 15:16:29,2015-04-27 15:31:48,1,2.89,-73.994118,40.756432,1,N,-73.993263,40.724442,1,12.5,0.0,0.5,3.32,0.0,0.3,16.62
2,2015-04-27 15:16:29,2015-04-27 15:25:26,1,0.94,-73.974388,40.761665,1,N,-73.985542,40.756168,2,7.5,0.0,0.5,0.00,0.0,0.3,8.30
2,2015-04-27 15:16:29,2015-04-27 15:45:38,1,2.30,-73.972183,40.762726,1,N,-73.990845,40.750671,2,17.5,0.0,0.5,0.00,0.0,0.3,18.30


In [3]:
df.reset_index(inplace=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-04-27 15:16:29,2015-04-27 15:26:14,1,1.05,-73.988869,40.758396,1,N,-73.984779,40.748272,2,7.5,0.0,0.5,0.00,0.0,0.3,8.30
1,2,2015-04-27 15:16:29,2015-04-27 15:25:03,1,2.67,-74.008568,40.704479,1,N,-73.978271,40.720543,1,10.5,0.0,0.5,1.00,0.0,0.3,12.30
2,2,2015-04-27 15:16:29,2015-04-27 15:31:48,1,2.89,-73.994118,40.756432,1,N,-73.993263,40.724442,1,12.5,0.0,0.5,3.32,0.0,0.3,16.62
3,2,2015-04-27 15:16:29,2015-04-27 15:25:26,1,0.94,-73.974388,40.761665,1,N,-73.985542,40.756168,2,7.5,0.0,0.5,0.00,0.0,0.3,8.30
4,2,2015-04-27 15:16:29,2015-04-27 15:45:38,1,2.30,-73.972183,40.762726,1,N,-73.990845,40.750671,2,17.5,0.0,0.5,0.00,0.0,0.3,18.30


## clean data

In [4]:
df = df.drop(['VendorID', 'trip_distance', 'improvement_surcharge',
         'RateCodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
         'tip_amount', 'tolls_amount', 'total_amount'], axis=1)
df.dropna(inplace=True)
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2015-04-27 15:16:29,2015-04-27 15:26:14,1,-73.988869,40.758396,-73.984779,40.748272
1,2015-04-27 15:16:29,2015-04-27 15:25:03,1,-74.008568,40.704479,-73.978271,40.720543
2,2015-04-27 15:16:29,2015-04-27 15:31:48,1,-73.994118,40.756432,-73.993263,40.724442
3,2015-04-27 15:16:29,2015-04-27 15:25:26,1,-73.974388,40.761665,-73.985542,40.756168
4,2015-04-27 15:16:29,2015-04-27 15:45:38,1,-73.972183,40.762726,-73.990845,40.750671


#### borders of NY City
city_long_border = (-74.03, -73.75);
city_lat_border = (40.63, 40.85) 

In [5]:
df = df[df['pickup_longitude'] <= -73.68]
df = df[df['pickup_longitude'] >= -74.27]
df = df[df['pickup_latitude'] <= 40.93]
df = df[df['pickup_latitude'] >= 40.45]
df = df[df['dropoff_longitude'] <= -73.68]
df = df[df['dropoff_longitude'] >= -74.27]
df = df[df['dropoff_latitude'] <= 40.93]
df = df[df['dropoff_latitude'] >= 40.45]
df.shape

(12822521, 7)

#### extract all the time features out

In [6]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y/%m/%d %H:%M:%S')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y/%m/%d %H:%M:%S')

In [7]:
df['year'] = df['tpep_pickup_datetime'].dt.year
df['month'] = df['tpep_pickup_datetime'].dt.month
df['hour'] = df['tpep_pickup_datetime'].dt.hour
df['dayofweek'] = df['tpep_pickup_datetime'].dt.weekday_name
df['year_quarter'] = df['tpep_pickup_datetime'].dt.quarter
df['dayofyear'] = df['tpep_pickup_datetime'].dt.dayofyear

#### delete trips beyond reasonable time range

In [8]:
df['timerange'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [9]:
df['timerange'] = df['timerange'].dt.seconds

In [10]:
#12 hours: 12*60*60=43200
df = df[df['timerange'] > 10]
df = df[df['timerange'] < 43200]

In [11]:
#this column is generated for merging with FHV data
df['day_and_hour'] = df['dayofyear'].astype(str) + '-' + df['hour'].astype(str)

#### For the holiday:
#### I tried to extract whether certain day is the day before a holiday, which I expect lots of people will rush to airport. However I fail to use the 'Holidays / Holiday Calendars' class in pandas which contains 'USFederalHolidayCalendar'.

# Geohash

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'year', 'month', 'year_quarter'], axis=1)

In [14]:
#add a column of geohash to dataframe
geo = {}
for i in range(len(df)):
    value = ghh.encode(df.dropoff_longitude.iloc[i], df.dropoff_latitude.iloc[i], precision=5)
    geo[i] = value
s = pd.Series(geo, name='geohash_dropoff')
geo = pd.DataFrame(s)
df = df.merge(geo, left_index=True, right_index=True)
df.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofweek,dayofyear,timerange,day_and_hour,geohash_dropoff
0,1,-73.988869,40.758396,-73.984779,40.748272,15,Monday,117,585,117-15,SHG1c
1,1,-74.008568,40.704479,-73.978271,40.720543,15,Monday,117,514,117-15,SHG2R
2,1,-73.994118,40.756432,-73.993263,40.724442,15,Monday,117,919,117-15,SHG2O
3,1,-73.974388,40.761665,-73.985542,40.756168,15,Monday,117,537,117-15,SHG1Z
4,1,-73.972183,40.762726,-73.990845,40.750671,15,Monday,117,1749,117-15,SHG1b


In [15]:
destination = {}
airportlist = ['SHG0t', 'SHFkk', 'SHGTe']
for i, geohash in enumerate(df['geohash_dropoff']):
    if geohash in airportlist:
        destination[i] = 1
    else:
        destination[i] = 0

In [16]:
s = pd.Series(destination, name='airport_as_destination')
destination = pd.DataFrame(s)
df1 = df.merge(destination, left_index=True, right_index=True)
df1.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofweek,dayofyear,timerange,day_and_hour,geohash_dropoff,airport_as_destination
0,1,-73.988869,40.758396,-73.984779,40.748272,15,Monday,117,585,117-15,SHG1c,0
1,1,-74.008568,40.704479,-73.978271,40.720543,15,Monday,117,514,117-15,SHG2R,0
2,1,-73.994118,40.756432,-73.993263,40.724442,15,Monday,117,919,117-15,SHG2O,0
3,1,-73.974388,40.761665,-73.985542,40.756168,15,Monday,117,537,117-15,SHG1Z,0
4,1,-73.972183,40.762726,-73.990845,40.750671,15,Monday,117,1749,117-15,SHG1b,0


In [17]:
len(df1[df1['airport_as_destination']==1])

65707

In [18]:
df1 = df1[df1['airport_as_destination']==1]

In [19]:
df1.reset_index(drop=True, inplace=True)

In [20]:
#add a column of geohash to dataframe
geo = {}
for i in range(len(df1)):
    value = ghh.encode(df1.pickup_longitude.iloc[i], df1.pickup_latitude.iloc[i], precision=5)
    geo[i] = value
s = pd.Series(geo, name='geohash_pickup')
geo = pd.DataFrame(s)
df1 = df1.merge(geo, left_index=True, right_index=True)
df1.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,hour,dayofweek,dayofyear,timerange,day_and_hour,geohash_dropoff,airport_as_destination,geohash_pickup
0,2,-73.967766,40.756351,-73.791580,40.644855,13,Friday,93,2152,93-13,SHFkk,1,SHG1V
1,1,-73.972321,40.786446,-73.872467,40.774338,5,Monday,96,1328,96-5,SHG0t,1,SHBJ4
2,1,-73.945976,40.777447,-73.872719,40.774303,5,Monday,96,763,96-5,SHG0t,1,SHG1D
3,2,-73.986458,40.758862,-73.872360,40.774483,10,Friday,100,1933,100-10,SHG0t,1,SHG1Z
4,1,-73.975967,40.749916,-73.792473,40.644768,19,Thursday,113,2047,113-19,SHFkk,1,SHG1X


In [21]:
df1.drop(['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'hour', 'geohash_dropoff', 'timerange'], axis=1, inplace=True)

In [22]:
#this column is generated for grouping
df1['geo_and_day_and_hour'] = df1['geohash_pickup'].astype(str) + df1['day_and_hour'].astype(str)
df1.head()

,dayofweek,dayofyear,day_and_hour,airport_as_destination,geohash_pickup,geo_and_day_and_hour
0,Friday,93,93-13,1,SHG1V,SHG1V93-13
1,Monday,96,96-5,1,SHBJ4,SHBJ496-5
2,Monday,96,96-5,1,SHG1D,SHG1D96-5
3,Friday,100,100-10,1,SHG1Z,SHG1Z100-10
4,Thursday,113,113-19,1,SHG1X,SHG1X113-19


In [23]:
df2 = df1.groupby('geo_and_day_and_hour', as_index=False).agg({'dayofweek':'first', 'dayofyear':'first',
                                        'day_and_hour':'first', 'airport_as_destination':'sum',
                                        'geohash_pickup':'first'})

In [24]:
df2.drop(['geo_and_day_and_hour'], axis=1, inplace=True)

In [25]:
df2.rename(columns={"airport_as_destination": "pickups"}, inplace=True)

In [26]:
df2.head()

,dayofweek,dayofyear,day_and_hour,pickups,geohash_pickup
0,Friday,100,100-11,1,SHBJ3
1,Friday,100,100-15,1,SHBJ3
2,Friday,100,100-6,1,SHBJ3
3,Sunday,102,102-12,1,SHBJ3
4,Sunday,102,102-6,1,SHBJ3


In [27]:
df2.shape

(26869, 5)

In [28]:
df2.to_csv('data/agg_clean_taxi_data_04.csv')